# Pre-processing of a word-based corpus

Author: Anne-Flore Cabouat

March 2023

In [155]:
import pandas as pd
import numpy as np
import re
import csv

## Your file

In [154]:
'''
#study questionnaire papers variables
prefix = "studies_questions"
corpus_Col = "keyword"
original_Col = "terms to keep" #if pre-processed keywords, this is the col with original content
source_Col = "doi link"
'''
#users comments variables
corpus_Col = "Keywords"
original_Col = "Users comments" #if pre-processed keywords, this is the col with original content
source_Col = "DOI"
prefix = "users_comments"


#your file path
path_in = "../input/"
filename = f"collected_from_{prefix}.csv"


#processing variables
token_Col = 'tokens'
statement_index_key = 'S_Index' # will hold the index of original_col_name
original_key = 'Originals' # will hold the content of original_col_name
source_key = 'Sources' # will hold the content of source_ID_col
tokens_key = 'Tokens_originals' # will hold the original tokens in stem and CONCEPT lexicons

#your results path
path_out = f"../results/tokenized + stemmed contents/from_{prefix}"

csv_file = path_in+filename
print(csv_file)

../input/collected_from_users_comments.csv


### File cleaning

In [60]:
#clean rogue newlines in csv (result of Linux/Windows mixed system use)
ANSI = "cp1252"

OldFile=csv_file
NewFile=f'{path_out}/{filename[:-4]}_cleaned.csv'

#reading it in as binary keeps the cr lf in windows as is
with (
    open(OldFile, 'rb') as f_in,
    open(NewFile, 'wb') as f_out,
):

    FileContent = f_in.read()

    #removing all line breaks including the ones after the carriage return
    oldLineFeed = b'\n'
    newLineFeed = b''

    FileContent = FileContent.replace(oldLineFeed, newLineFeed)

    #only have a carriage return now at the end of each true line, added back in the line break 
    oldLineFeed = b'\r'
    newLineFeed = b'\r\n'
    FileContent = FileContent.replace(oldLineFeed, newLineFeed)

    f_out.write(FileContent)

f_in.close()
f_out.close()

csv_file = NewFile

'#clean rogue newlines in csv\nANSI = "cp1252"\n\nOldFile=csv_file\nNewFile=f\'{path_out}/{filename[:-4]}_cleaned.csv\'\n\n#reading it in as binary keeps the cr lf in windows as is\nwith (\n    open(OldFile, \'rb\') as f_in,\n    open(NewFile, \'wb\') as f_out,\n):\n\n    FileContent = f_in.read()\n\n    #removing all line breaks including the ones after the carriage return\n    oldLineFeed = b\'\n\'\n    newLineFeed = b\'\'\n\n    FileContent = FileContent.replace(oldLineFeed, newLineFeed)\n\n    #only have a carriage return now at the end of each true line, added back in the line break \n    oldLineFeed = b\'\r\'\n    newLineFeed = b\'\r\n\'\n    FileContent = FileContent.replace(oldLineFeed, newLineFeed)\n\n    f_out.write(FileContent)\n\nf_in.close()\nf_out.close()\n\ncsv_file = NewFile\n'

## Import

### Import/export cleaning functions 

In [61]:
# flattened list (str) back to list (list)
def str_back_to_list(my_str):
    if isinstance(my_str, str) and my_str[0]=='[':
        #print('Found flattened list')
        bool_switch=True
        my_str_list = my_str.strip('][').split(', ')
        my_list = [item[1:-1] if not item[0].isalnum() else item for item in my_str_list] #we assume non alpha means unwanted ' ' around the token and strip them from each item
        #print(type(my_list), len(my_list))
        return my_list
    else:
        return my_str

#to change numpy integer into regular int
def change_int_type(value):
    if isinstance(value, np.integer):
        #print("found numpy int")
        return(value.item())
    elif isinstance(value, int):
        #print("found int")
        return(int(value))
    else:
        return(value)

def clean_my_imported_object(my_object):
    my_object = str_back_to_list(my_object)
    my_object = change_int_type(my_object)
    return my_object


def clean_my_imported_csv(df):
    for index, row in df.iterrows():
        for col in list(df.columns):
            df.at[index, col] = clean_my_imported_object(df.at[index, col])
    return df
            

### Initializing corpus df

In [62]:
#prepare DataFrame & subset
def get_data_ready(my_csv, cols):
    df = clean_my_imported_csv(pd.read_csv(my_csv))
    df_corpus = pd.DataFrame(columns=cols)
    print(df_corpus)
    df_corpus
    for col in cols:
        #delete \n both for windwos r\n and Unix \n systems
        for txt in df[col]:
            if isinstance(txt,str):
                re.sub('\r?\n', '###', txt)
            else:
                txt = ''
        df_corpus[col] = df[col]
    """
    try:
        global all_corpus
        all_corpus = df[col].to_list() #creating a list of strings
    except:
        print(f"Something went wrong with the choice of column '{col}'")
        pass
    df_corpus['corpus list'] = all_corpus
    print(f'----------------------\nCORPUS FIRST AND LAST\n----------------------\n{all_corpus[0]}\n\n(...)\n\n{all_corpus[-1]}')
    """
    return df_corpus

In [63]:

df_corpus_init = get_data_ready(csv_file, [corpus_Col,original_Col,source_Col])
df_corpus_init

Empty DataFrame
Columns: [Keywords, Users comments, DOI]
Index: []


,Keywords,Users comments,DOI
0,readable; visible,our users who rightly noted that only one leve...,https://dx.doi.org/10.1109/INFVIS.2002.1173148
1,legible;,Suggestions for improvement had several themes...,https://dx.doi.org/10.1109/TVCG.2009.176
2,readable; simple;,The continuous timeline requires more space fo...,https://dx.doi.org/10.1109/VAST.2009.5332595
3,better; suitable; difficult; follow; lost,We also collected written comments from our pa...,https://dx.doi.org/10.1109/TVCG.2011.186
4,decipher; confusing;,Participants’ feedback from the questionnaire ...,https://dx.doi.org/10.1109/TVCG.2011.160
5,rich; usable; comprehensibly; easily; simplici...,The major benefits of RelEx most frequently st...,https://dx.doi.org/10.1109/TVCG.2012.255
6,meaningful; view; simple; understand; helpful,Based on free response questions and interview...,https://dx.doi.org/10.1109/TVCG.2012.189
7,difficult;,We categorized the difficult aspects described...,https://dx.doi.org/10.1109/TVCG.2013.151
8,readability; easy to see; easy to read; legibl...,Six audience participants chose PowerPoint as ...,https://dx.doi.org/10.1109/TVCG.2013.191
9,easy; see; difficult; clutter; use;,"Comparing diff to side-by-side, PD08 said “it’...",https://dx.doi.org/10.1109/TVCG.2013.233


In [65]:
#possible output for manual check
#df_corpus_init.to_csv(f'{path_out}/csv_init_verif.csv')

#drop na and compare the end of df (index length)
df_corpus_init = df_corpus_init.dropna()
df_corpus_init.tail(10)

,Keywords,Users comments,DOI
25,intuitively; easy; hamper; useful; appropriate...,Visual Design.\n\nMost of the users agreed tha...,https://dx.doi.org/10.1109/TVCG.2018.2835485
26,complex;,Eiffel facilitates the analysis of static infl...,https://dx.doi.org/10.1109/TVCG.2019.2906900
27,simple; complex;,The majority of the participants pre- ferred C...,https://dx.doi.org/10.1109/TVCG.2020.2968911
28,distracting; messy; cluttered; disorganized; u...,"Another participant commented, “Left graph use...",https://dx.doi.org/10.1109/TVCG.2021.3068337
29,useful; insight; easily; understand; prefer; u...,“I feel like this is providing useful insight ...,https://dx.doi.org/10.1109/TVCG.2021.3114822
30,balance;,The qualitative feedback showed that people ap...,https://dx.doi.org/10.1109/TVCG.2021.3114775
31,understand; interpretation;,"TAM also received two neutral assessments, jus...",https://dx.doi.org/10.1109/TVCG.2022.3209477
32,occlusion; balance,The usability is high because mixed layout ‘ma...,https://dx.doi.org/10.1109/TVCG.2022.3209475
33,"intuitive; easy, understand;",We got comments like “Interface is user friend...,https://dx.doi.org/10.1109/TVCG.2022.3209484
34,easy; hard; confusing; visible; see; crowded; ...,"comments sorted under ""Visibility"" (from addit...",https://dx.doi.org/10.1109/TVCG.2022.3209480


### Count the sources and the original contents

In [66]:
sources_counts_df = pd.DataFrame(df_corpus_init[source_Col].explode().dropna().value_counts())

#df_corpus_originals_count = df_corpus_init.groupby(['doi link','terms to keep'])['terms to keep'].count()
originals_counts_df = df_corpus_init.groupby([source_Col,original_Col]).size().reset_index(name='counts') #https://stackoverflow.com/questions/19384532/get-statistics-for-each-group-such-as-count-mean-etc-using-pandas-groupby

sources_nb = len(sources_counts_df)
print(f'{sources_nb} sources')

originals_nb =  len(originals_counts_df['counts'])
#originals_nb =  originals_counts_df['counts'].sum().item()
print(f'{originals_nb} originals')

35 sources
35 originals


In [67]:
#verif unique in list
sources_list=sources_counts_df.index.values.tolist()
print(len(sources_list))
print(len(set(sources_list)))

#save as csv (to check intersection with other corpus)
pd.DataFrame(sources_counts_df.index).to_csv(f'{path_out}/sources_unique_list.csv')

35
35


In [68]:
originals_counts_df

,DOI,Users comments,counts
0,https://dx.doi.org/10.1109/INFVIS.2002.1173148,our users who rightly noted that only one leve...,1
1,https://dx.doi.org/10.1109/TVCG.2009.176,Suggestions for improvement had several themes...,1
2,https://dx.doi.org/10.1109/TVCG.2011.160,Participants’ feedback from the questionnaire ...,1
3,https://dx.doi.org/10.1109/TVCG.2011.186,We also collected written comments from our pa...,1
4,https://dx.doi.org/10.1109/TVCG.2012.189,Based on free response questions and interview...,1
5,https://dx.doi.org/10.1109/TVCG.2012.255,The major benefits of RelEx most frequently st...,1
6,https://dx.doi.org/10.1109/TVCG.2013.151,We categorized the difficult aspects described...,1
7,https://dx.doi.org/10.1109/TVCG.2013.191,Six audience participants chose PowerPoint as ...,1
8,https://dx.doi.org/10.1109/TVCG.2013.233,"Comparing diff to side-by-side, PD08 said “it’...",1
9,https://dx.doi.org/10.1109/TVCG.2013.76,participants commented that the linear design ...,1


## Tokenize words

### Import libraries

In [69]:
import nltk
import numpy as np

#contractions changes words like "can't" to "cannot" => english text will be isalpha()
import contractions

from nltk import word_tokenize

#we use nltk's stopwords
from nltk.corpus import stopwords

#and for stemming:
from nltk import stem
from nltk.stem import SnowballStemmer
from nltk.stem.porter import *

### For custom stop words

In [70]:
#if you have issues with nltk ssl, run this cell (MacOS)
'''
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

'''

'\nimport ssl\n\ntry:\n    _create_unverified_https_context = ssl._create_unverified_context\nexcept AttributeError:\n    pass\nelse:\n    ssl._create_default_https_context = _create_unverified_https_context\n\n'

In [71]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aviz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aviz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [72]:
#print(stopwords.words('english'))

new_stopwords = [
    '“','”','...','(',')','[',']','{','}','++',
    '+<p>','+<head>','+<ref>','+<figDesc>', '+<title>',
    '<p>', '</p>','<head>','<ref>','<figDesc>', '<title>',
    'p', '<', '>',
    ',','.','?','!', ':', ';',
]
stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(new_stopwords)
print(stpwrd)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Remove stopwords & tokenize functions

In [74]:
def remove_stop_words(text_tokens):
    if isinstance(text_tokens[0],list) and len(text_tokens) == 1:
        text_tokens = text_tokens[0]
    elif isinstance(text_tokens[0],list) and len(text_tokens) > 1:
        unnested_list = []
        for item in text_tokens:
            unnested_list += item
        text_tokens = unnested_list
    text_tokens = list(map(lambda w : w.lower(), text_tokens))
    text_tokens = [word for word in text_tokens if word.isalpha() and not word in stpwrd]
    return text_tokens

def word_tokenize_col(df, col):
    df[token_Col]=''
    global row_tokens
    global unique_tokens
    global word_count
    unique_tokens = []
    word_count = 0
    for index, row in df.iterrows():
        cell = str(df.at[index, col])
        contractions.fix(cell)
        row_tokens = []
        df.at[index, token_Col] = []
        row_tokens.append(word_tokenize(cell))
        row_tokens = remove_stop_words(row_tokens)
        df.at[index, token_Col] = row_tokens
        word_count += len(row_tokens)
        unique_tokens = unique_tokens + [t for t in row_tokens if t not in unique_tokens]
        
    return df

### Tokenize corpus

In [73]:
#check for str in corpus
if all(isinstance(s, str) for s in df_corpus_init[corpus_Col]):
    print('OK: only strings in corpus')

OK: only strings in corpus


In [75]:
df_corpus = []
df_corpus = word_tokenize_col(df_corpus_init, corpus_Col)
print(f'Total of words: {word_count}')
print(f'Unique words: {len(unique_tokens)}')
print(unique_tokens)

Total of words: 156
Unique words: 92
['readable', 'visible', 'legible', 'simple', 'better', 'suitable', 'difficult', 'follow', 'lost', 'decipher', 'confusing', 'rich', 'usable', 'comprehensibly', 'easily', 'simplicity', 'speed', 'understandable', 'meaningful', 'view', 'understand', 'helpful', 'readability', 'easy', 'see', 'easy', 'read', 'ease', 'visibility', 'clean', 'bold', 'untidy', 'cluttered', 'clutter', 'use', 'occluded', 'recognize', 'guide', 'intuitive', 'strong', 'judge', 'clear', 'disorientating', 'unexpected', 'distinguish', 'encoding', 'sparse', 'overview', 'instantly', 'interpret', 'hard', 'cognitive', 'present', 'perceive', 'coding', 'obvious', 'effective', 'discern', 'unclear', 'meaning', 'correctly', 'spot', 'visualize', 'conceptualize', 'distracting', 'chaos', 'easier', 'busy', 'messy', 'comfortable', 'richer', 'neat', 'intuitively', 'hamper', 'useful', 'appropriate', 'efficient', 'highlight', 'attract', 'attention', 'complex', 'disorganized', 'unappealing', 'insight',

C:\Users\Aviz\AppData\Local\Temp\ipykernel_16032\3730504777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[token_Col]=''


In [76]:
df_corpus

,Keywords,Users comments,DOI,tokens
0,readable; visible,our users who rightly noted that only one leve...,https://dx.doi.org/10.1109/INFVIS.2002.1173148,"[readable, visible]"
1,legible;,Suggestions for improvement had several themes...,https://dx.doi.org/10.1109/TVCG.2009.176,[legible]
2,readable; simple;,The continuous timeline requires more space fo...,https://dx.doi.org/10.1109/VAST.2009.5332595,"[readable, simple]"
3,better; suitable; difficult; follow; lost,We also collected written comments from our pa...,https://dx.doi.org/10.1109/TVCG.2011.186,"[better, suitable, difficult, follow, lost]"
4,decipher; confusing;,Participants’ feedback from the questionnaire ...,https://dx.doi.org/10.1109/TVCG.2011.160,"[decipher, confusing]"
5,rich; usable; comprehensibly; easily; simplici...,The major benefits of RelEx most frequently st...,https://dx.doi.org/10.1109/TVCG.2012.255,"[rich, usable, comprehensibly, easily, simplic..."
6,meaningful; view; simple; understand; helpful,Based on free response questions and interview...,https://dx.doi.org/10.1109/TVCG.2012.189,"[meaningful, view, simple, understand, helpful]"
7,difficult;,We categorized the difficult aspects described...,https://dx.doi.org/10.1109/TVCG.2013.151,[difficult]
8,readability; easy to see; easy to read; legibl...,Six audience participants chose PowerPoint as ...,https://dx.doi.org/10.1109/TVCG.2013.191,"[readability, easy, see, easy, read, legible, ..."
9,easy; see; difficult; clutter; use;,"Comparing diff to side-by-side, PD08 said “it’...",https://dx.doi.org/10.1109/TVCG.2013.233,"[easy, see, difficult, clutter, use]"


### Create token dict: the lexicon

In [78]:
all_tokens_df = df_corpus.explode(token_Col)
all_tokens_df.set_index(token_Col, inplace=True)
all_tokens_df

df_cols = list(all_tokens_df)

token_lexicon = {}

for token, row in all_tokens_df.iterrows():
    #if first time, prepare the structure
    if token not in token_lexicon.keys():
        token_lexicon.update({
            token:{}
        })
        for col in df_cols:
            token_lexicon[token].update({
                col:[]
            })
    #add this row elements to the values
    for col in df_cols:
        old_value = token_lexicon[token][col]
        token_lexicon[token].update({
            col: old_value+[row[col]]
        })

simple_tokens_count = {}
for token in token_lexicon:
    this_token_source_count = len(set(token_lexicon[token][source_Col]))
    this_token_originals_count = len(token_lexicon[token][original_Col])
    token_lexicon[token].update({
        f'{source_key}_count':  this_token_source_count,
        f'{original_key}_count': this_token_originals_count
    })

    simple_tokens_count.update({
        token:this_token_source_count
    })



In [79]:
#check count dict
simple_tokens_count['read']

1

In [80]:
all_tokens_count = pd.DataFrame.from_dict(simple_tokens_count, orient='index', columns=['count'])
all_tokens_count


,count
readable,2
visible,2
legible,2
simple,4
better,1
...,...
balance,2
interpretation,1
occlusion,1
crowded,1


In [81]:
all_tokens_count.to_csv(f'{path_out}/{prefix}-all_tokens_count.csv')

### Count word tokens

In [82]:
tokens_count = all_tokens_count['count'].sum().item()
print(f'{tokens_count} tokens')
unique_tokens = len(all_tokens_count)
print(f'{unique_tokens} unique tokens')

154 tokens
91 unique tokens


## Stem the tokens

### Lexicon (dict) functions

In [83]:

#df from lexicon    
def df_from_lexicon(lexicon_dict, keep_cols=[], excl_cols=[]):
    df_lexicon = pd.DataFrame.from_dict(lexicon_dict, orient='index')
    df_lexicon = df_lexicon.reset_index()
    df_cols = list(df_lexicon.columns)
    
    if not isinstance(keep_cols, list):
        raise Exception('keep_cols must be a list')        
    if not isinstance(excl_cols, list):
        raise Exception('excl_cols must be a list')
    
    my_cols_to_keep=[]
    if len(keep_cols)>0 and len(excl_cols)>0:
        raise Exception('Choose either keep_cols or excl_cols, not both')
    elif len(excl_cols)>0:
        my_cols_to_keep = [col for col in df_cols if col not in excl_cols]
        df_lexicon.filter(items=my_cols_to_keep)
    elif len(keep_cols)>0:
        my_cols_to_keep = keep_cols
        df_lexicon.filter(items=my_cols_to_keep)
        
    return df_lexicon


#Some density calculations

#function to unpack numerators and denominators
def unpack_arg_tuple(lexicon,my_arg,k):
    if isinstance(my_arg, tuple):
        if my_arg[0] == 'subkey' and len(my_arg) == 2:
            return lexicon[k][my_arg[1]]
        else:
            print(f'{my_arg} should be a tuple of two elements: "subkey" indicatore and the lexicon key to reach')

    elif isinstance(my_arg, int) or isinstance(my_arg, float):
        return my_arg #component can be used as such

    else:
        print(f'{type(my_arg)}')
        raise Exception(f'{my_arg} is neither a tuple nor a number')
        

# Make calculations
def density_in_lexicon(lexicon, new_key_for_density, numerator, denominator):
    arg_to_unpack_list = [numerator,denominator]
    for k in lexicon.keys():
        fraction_components = []
        for component in arg_to_unpack_list:
            fraction_components.append(unpack_arg_tuple(lexicon, component, k))

        lexicon[k].update({new_key_for_density:fraction_components[0]/fraction_components[1]})
        
    return lexicon

### Stemming functions

In [85]:
#choose a language
stemmer = SnowballStemmer("english")
#the lexicon will hold all original tokens to be added on the last dataframe

#Stemming from corpus
def wStem(text):
    #stemming
    stemmed_text = stemmer.stem(text)
    #updating the lexicon
    global lexicon
    
    ##if new stemmed token
    if stemmed_text not in lexicon:
        lexicon[stemmed_text] = [text]
    
    ##stemmed token already in lexicon BUT the original text is not
    elif stemmed_text in lexicon and text not in lexicon[stemmed_text]:
        old_list = lexicon[stemmed_text]
        new_list = old_list + [text]
        lexicon.update({stemmed_text:new_list})
    else:
        pass
    return stemmed_text

#stemming from pre-processed list of keywords with original text in different col
def kStem(keyword, df, cols_keys_tuples_list, current_index):
    #stemming
    stemmed_keyword = stemmer.stem(keyword)
    #print(f'stem = {stemmed_keyword}')
    
    #updating the lexicon
    global lexicon_stems
    
    for my_tuple in cols_keys_tuples_list:
        my_col = my_tuple[0]
        my_key = my_tuple[1]
        new_content = df.at[current_index, my_col]
        #print(f'New content is {new_content}')
        
        #turn lists into strings
#        if isinstance(new_content, list) and len(new_content)>0:
#            new_content = ' '.join(new_content)

        ##if stemmed token is new
        if stemmed_keyword not in lexicon_stems.keys():
            lexicon_stems.update({stemmed_keyword:{
                my_key:[new_content],
                tokens_key:[keyword],
                }
            })

            #print(f'New stem added: {stemmed_keyword}')

        ##elif stemmed token is already in lexicon BUT the new key text is not
        elif my_key not in lexicon_stems[stemmed_keyword].keys():
            lexicon_stems[stemmed_keyword].update({my_key:[new_content]})
            #print(f'New content added to stem: {new_content}')

        ##elif stemmed token is already in lexicon and also the current key BUT the content is new
        elif my_key in lexicon_stems[stemmed_keyword].keys():
            content_list = lexicon_stems[stemmed_keyword][my_key]
            new_content_list = content_list+[new_content]
            lexicon_stems[stemmed_keyword].update({my_key:new_content_list})
            #print(f'Added "{new_content}" to "{stemmed_keyword}" entry.')
        else:
            print('!!! Something weird')
            pass
        
        #finally add the token if it's new
        if keyword not in lexicon_stems[stemmed_keyword][tokens_key]:
            old_tokens_list = lexicon_stems[stemmed_keyword][tokens_key]
            old_tokens_list += [keyword]
            lexicon_stems[stemmed_keyword].update({tokens_key:old_tokens_list})
            
    return stemmed_keyword

In [86]:
#using stem on a df[col] content made from tokenized keywords
def stem_keyword_col(df, keyword_col, keep_cols_keys_tuples_list):
    df['stem tokens']='' #initiate new column to store stemmed tokens
    #global row_stem_tokens
    global unique_stem_tokens
    global stem_word_count
    unique_stem_tokens = []
    for index, row in df.iterrows():
        cell = df.at[index, keyword_col]
        row_stem_tokens=[]
        row_stem_tokens = list(map(lambda k : kStem(k, df, keep_cols_keys_tuples_list, index), cell))
        df.at[index, 'stem tokens'] = row_stem_tokens
        unique_stem_tokens = unique_stem_tokens + [t for t in row_stem_tokens if t not in unique_stem_tokens]        
    return df

### Stemming the corpus

In [88]:
lexicon = {}
lexicon_stems = {}

''' THIS IS A REMINDER OF VARIABLES DECLARED IN CELL #2
token_Col = 'tokens'
statement_index_key = 'S_Index' # will hold the index of original_col_name
original_key = 'Originals' # will hold the content of original_col_name
source_key = 'Sources' # will hold the content of source_ID_col
tokens_key = 'Tokens_originals' # will hold the original tokens in stem and CONCEPT lexicons
'''

cols_to_keep_and_keys =[
    (corpus_Col, "Selected keywords in original"),
    (original_Col, original_key),
    (source_Col,source_key)
]
df_corpus = stem_keyword_col(df_corpus, token_Col, cols_to_keep_and_keys)
print(f'Length of lexicon_stems: {len(lexicon_stems)}')

Length of lexicon_stems: 84


C:\Users\Aviz\AppData\Local\Temp\ipykernel_16032\973663967.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stem tokens']='' #initiate new column to store stemmed tokens


In [89]:
df_corpus

,Keywords,Users comments,DOI,tokens,stem tokens
0,readable; visible,our users who rightly noted that only one leve...,https://dx.doi.org/10.1109/INFVIS.2002.1173148,"[readable, visible]","[readabl, visibl]"
1,legible;,Suggestions for improvement had several themes...,https://dx.doi.org/10.1109/TVCG.2009.176,[legible],[legibl]
2,readable; simple;,The continuous timeline requires more space fo...,https://dx.doi.org/10.1109/VAST.2009.5332595,"[readable, simple]","[readabl, simpl]"
3,better; suitable; difficult; follow; lost,We also collected written comments from our pa...,https://dx.doi.org/10.1109/TVCG.2011.186,"[better, suitable, difficult, follow, lost]","[better, suitabl, difficult, follow, lost]"
4,decipher; confusing;,Participants’ feedback from the questionnaire ...,https://dx.doi.org/10.1109/TVCG.2011.160,"[decipher, confusing]","[deciph, confus]"
5,rich; usable; comprehensibly; easily; simplici...,The major benefits of RelEx most frequently st...,https://dx.doi.org/10.1109/TVCG.2012.255,"[rich, usable, comprehensibly, easily, simplic...","[rich, usabl, comprehens, easili, simplic, spe..."
6,meaningful; view; simple; understand; helpful,Based on free response questions and interview...,https://dx.doi.org/10.1109/TVCG.2012.189,"[meaningful, view, simple, understand, helpful]","[meaning, view, simpl, understand, help]"
7,difficult;,We categorized the difficult aspects described...,https://dx.doi.org/10.1109/TVCG.2013.151,[difficult],[difficult]
8,readability; easy to see; easy to read; legibl...,Six audience participants chose PowerPoint as ...,https://dx.doi.org/10.1109/TVCG.2013.191,"[readability, easy, see, easy, read, legible, ...","[readabl, easi, see, easi, read, legibl, eas, ..."
9,easy; see; difficult; clutter; use;,"Comparing diff to side-by-side, PD08 said “it’...",https://dx.doi.org/10.1109/TVCG.2013.233,"[easy, see, difficult, clutter, use]","[easi, see, difficult, clutter, use]"


In [90]:
#checking the dict
lexicon_stems["easi"]

{'Selected keywords in original': ['readability; easy to see; easy to read; legible; ease of visibility; understand; clean; bold; untidy; cluttered;',
  'readability; easy to see; easy to read; legible; ease of visibility; understand; clean; bold; untidy; cluttered;',
  'easy; see; difficult; clutter; use;',
  'strong; easy;',
  'easy; difficult; unexpected; clear; distinguish; encoding; intuitive; sparse;',
  'easy; understand; meaningful; present (?);',
  'easy; perceive; helpful; distinguish; coding; obvious;  effective',
  'difficult; understand; easy; visualize; conceptualize;',
  'intuitively; easy; hamper; useful; appropriate; confusing; distinguish; efficient; highlight; attract attention;',
  'intuitive; easy, understand;',
  'easy; hard; confusing; visible; see; crowded; convenient; show'],
 'Tokens_originals': ['easy'],
 'Originals': ['Six audience participants chose PowerPoint as a preferred system mainly because of the readability. They mentioned that PowerPoint was “easie

In [91]:
lexicon_stems["eas"]

{'Selected keywords in original': ['readability; easy to see; easy to read; legible; ease of visibility; understand; clean; bold; untidy; cluttered;',
  'judge; ease; clear;'],
 'Tokens_originals': ['ease'],
 'Originals': ['Six audience participants chose PowerPoint as a preferred system mainly because of the readability. They mentioned that PowerPoint was “easier to read,” “legible,” and “easier to see and read.” Another reason by two audience participants was familiarity. For example, P14 stated that “The ease of visibility understanding the flow, the cleanness of the graphics in each slide the boldness of the fonts much easier to follow along with it as I was more familiar with it.”\n(...)\nSome audience participants preferred PowerPoint because SketchStory looked untidy and cluttered for them. Furthermore, one major issue with SketchStory for the audience was readability; each chart did not have a title like a PowerPoint slide and the fonts for the numbers and labels were too small

In [92]:
#count the stems in all the originals of the corpus (for information only, not used in future calculations)
all_stemmed_tokens_count = pd.DataFrame()
all_stemmed_tokens_count['count'] = pd.DataFrame(df_corpus['stem tokens'].explode().dropna().value_counts())
stems_count = all_stemmed_tokens_count['count'].sum().item()
print(f'{stems_count} stems in all originals')
unique_stems = len(all_stemmed_tokens_count)
print(f'{unique_stems} unique stems in corpus')

156 stems in all originals
84 unique stems in corpus


In [93]:
lexicon_stems.keys()

dict_keys(['readabl', 'visibl', 'legibl', 'simpl', 'better', 'suitabl', 'difficult', 'follow', 'lost', 'deciph', 'confus', 'rich', 'usabl', 'comprehens', 'easili', 'simplic', 'speed', 'understand', 'meaning', 'view', 'help', 'easi', 'see', 'read', 'eas', 'clean', 'bold', 'untidi', 'clutter', 'use', 'occlud', 'recogn', 'guid', 'intuit', 'strong', 'judg', 'clear', 'disorient', 'unexpect', 'distinguish', 'encod', 'spars', 'overview', 'instant', 'interpret', 'hard', 'cognit', 'present', 'perceiv', 'code', 'obvious', 'effect', 'discern', 'unclear', 'mean', 'correct', 'spot', 'visual', 'conceptu', 'distract', 'chao', 'easier', 'busi', 'messi', 'comfort', 'richer', 'neat', 'hamper', 'appropri', 'effici', 'highlight', 'attract', 'attent', 'complex', 'disorgan', 'unapp', 'insight', 'prefer', 'relev', 'show', 'balanc', 'occlus', 'crowd', 'conveni'])

## Update lexicon with counts and densities

### Add tokens counts

In [94]:
for stem in lexicon_stems.keys():
    this_stem_tokens_list = lexicon_stems[stem][tokens_key]
    lexicon_stems[stem].update({
        tokens_key:{}
    })
    for token in this_stem_tokens_list:
        lexicon_stems[stem][tokens_key].update({
            token: token_lexicon[token][f'{source_key}_count']
        })

In [95]:
lexicon_stems['understand'][tokens_key]

{'understandable': 1, 'understand': 9}

### Count stems occurences in originals and sources

In [96]:
keys_to_count = [original_key, source_key]
for stem in lexicon_stems.keys():
    for key_to_count in keys_to_count:
        lexicon_stems[stem].update({
            f'{key_to_count}_count':len(lexicon_stems[stem][key_to_count])
        })

In [97]:
#check one
lexicon_stems['understand']['Originals_count']

10

### Density calculations

In [98]:
source_key='Sources'
original_key='Originals'

lexicon_stems = density_in_lexicon(
    lexicon_stems, #lexicon
    f'{source_key}_density', #new key to store the density in the lexicon
    ('subkey', f'{source_key}_count'), #numerator: number of sources who used the token
    sources_nb #denominator: total number of sources
)

#the density of token across statements
lexicon_stems = density_in_lexicon(
    lexicon_stems, #lexicon
    f'{original_key}_density', #new key to store the density in the lexicon
    ('subkey', f'{original_key}_count'), #numerator: token number of occurences across the corpus
    originals_nb #denominator: total number of original statements
)

#the density of sources in the use of this token
#number of source / total occurences 
##If a token has been heavily used but only by one person, this score will be low
##If a token has been by many people and they used it only once, this score will be high

lexicon_stems = density_in_lexicon(
    lexicon_stems, #lexicon
    f'{source_key}_use_density', #new key to store the density in the lexicon
    ('subkey', f'{source_key}_count'), #numerator: number of sources who used the token
    ('subkey', f'{original_key}_count') #denominator: token number of occurences across the corpus
)

In [99]:
lexicon_stems['understand']

{'Selected keywords in original': ['rich; usable; comprehensibly; easily; simplicity; speed; understandable;',
  'meaningful; view; simple; understand; helpful',
  'readability; easy to see; easy to read; legible; ease of visibility; understand; clean; bold; untidy; cluttered;',
  'understand; recognize; disorientating;',
  'easy; understand; meaningful; present (?);',
  'discern; interpret; understand;',
  'difficult; understand; easy; visualize; conceptualize;',
  'useful; insight; easily; understand; prefer; useful; see; relevant; show; attention; obvious',
  'understand; interpretation;',
  'intuitive; easy, understand;'],
 'Tokens_originals': {'understandable': 1, 'understand': 9},
 'Originals': ['The major benefits of RelEx most frequently stated by our participants were the overview RelEx provides, the rich and usable exploration opportunities, its comprehensibility, the ability to easily communicate with colleagues using RelEx, and the ability to speed up and simplify a variety

## Make df & output

In [100]:
df_stem_lexicon = df_from_lexicon(lexicon_stems).drop('Selected keywords in original', axis=1)

In [101]:
df_stem_lexicon

,index,Tokens_originals,Originals,Sources,Originals_count,Sources_count,Sources_density,Originals_density,Sources_use_density
0,readabl,"{'readable': 2, 'readability': 1}",[our users who rightly noted that only one lev...,[https://dx.doi.org/10.1109/INFVIS.2002.117314...,3,3,0.085714,0.085714,1.0
1,visibl,"{'visible': 2, 'visibility': 1}",[our users who rightly noted that only one lev...,[https://dx.doi.org/10.1109/INFVIS.2002.117314...,3,3,0.085714,0.085714,1.0
2,legibl,{'legible': 2},[Suggestions for improvement had several theme...,"[https://dx.doi.org/10.1109/TVCG.2009.176, htt...",2,2,0.057143,0.057143,1.0
3,simpl,{'simple': 4},[The continuous timeline requires more space f...,"[https://dx.doi.org/10.1109/VAST.2009.5332595,...",4,4,0.114286,0.114286,1.0
4,better,{'better': 1},[We also collected written comments from our p...,[https://dx.doi.org/10.1109/TVCG.2011.186],1,1,0.028571,0.028571,1.0
...,...,...,...,...,...,...,...,...,...
79,show,{'show': 2},[“I feel like this is providing useful insight...,"[https://dx.doi.org/10.1109/TVCG.2021.3114822,...",2,2,0.057143,0.057143,1.0
80,balanc,{'balance': 2},[The qualitative feedback showed that people a...,"[https://dx.doi.org/10.1109/TVCG.2021.3114775,...",2,2,0.057143,0.057143,1.0
81,occlus,{'occlusion': 1},[The usability is high because mixed layout ‘m...,[https://dx.doi.org/10.1109/TVCG.2022.3209475],1,1,0.028571,0.028571,1.0
82,crowd,{'crowded': 1},"[comments sorted under ""Visibility"" (from addi...",[https://dx.doi.org/10.1109/TVCG.2022.3209480],1,1,0.028571,0.028571,1.0


In [102]:
df_stem_lexicon.to_csv(f'{path_out}/stems_from_{prefix}.csv')